In [3]:
import torch, os
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision.models as models
from train_model import train_model
from test_model import test_model
%matplotlib inline

In [ ]:
## make subfolder for images

In [10]:
image_dir = '../tiny_classification/tiny-imagenet-200/val'
model = models.resnet18()
model.avgpool = nn.AdaptiveAvgPool2d(1)
model.fc.out_features = 200
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model.load_state_dict(torch.load('models/224/model_10_epoch.pt', map_location=device))
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)



data_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.4802, 0.4481, 0.3975], [0.2302, 0.2265, 0.2262])
])
image_dataset = datasets.ImageFolder(image_dir, data_transforms)
num_workers = 0
    
dataloader = data.DataLoader(image_dataset, batch_size=1, shuffle=False, num_workers=num_workers)

In [22]:
def inference(image_dir, pred_file):
    model = models.resnet18()
    model.avgpool = nn.AdaptiveAvgPool2d(1)
    model.fc.out_features = 200
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    model.load_state_dict(torch.load('models/224/model_10_epoch.pt', map_location=device))
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    
    model.eval()
    
    data_transforms = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize(224),
        transforms.Normalize([0.4802, 0.4481, 0.3975], [0.2302, 0.2265, 0.2262])
        
    ])
    
    
    image_dataset = datasets.ImageFolder(image_dir, data_transforms)
    num_workers = 0
    
    dataloader = data.DataLoader(image_dataset, batch_size=20, shuffle=False, num_workers=num_workers)
#     print(next(iter(dataloader)))
    
    predictions = []
    
    
    for i, (inputs, labels) in enumerate(dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        
        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            print(labels)
            # vectors = nn.AvgLayer(1, 64)
            preds = preds.data.cpu().numpy().flatten().tolist()
            predictions += preds
            print(preds)
            print()
    
    return predictions        

In [23]:
inference('../tiny_classification/tiny-imagenet-200/val', 'pred.csv')

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
[0, 0, 0, 0, 0, 0, 0, 0, 196, 152, 73, 0, 0, 152, 0, 0, 0, 0, 0, 0]

tensor([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
[0, 0, 0, 0, 0, 1, 1, 1, 19, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
[1, 1, 1, 1, 152, 1, 1, 1, 1, 196, 16, 2, 2, 3, 2, 3, 6, 4, 2, 3]

tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3])
[3, 39, 2, 3, 2, 16, 2, 7, 2, 196, 196, 2, 123, 2, 2, 3, 3, 39, 3, 3]



KeyboardInterrupt: 